# Converting between formats
In this section we are going to look at how to convert from one data format to another. There are many tools available for converting between formats, and we will use some of the most common ones: `samtools`, `bcftools` and `Picard`. Although it might not be obvious yet how this is useful, the tasks are a good way to become familiar with and understand the different option for the `samtools` and `bcftools`. 

## SAM/BAM
To convert between SAM and BAM formats you can use the __`samtools view`__ command.

To convert a BAM file to a SAM file try:

In [ ]:
samtools view -h NA20538.bam > NA20538.sam

Note we use the __`-h`__ option here to include the header in the SAM file. Now, have a look at the first five lines of the SAM file. 

In [ ]:
head -5 NA20538.sam

Well that was easy! And converting SAM to BAM is just as straightforward, try:

In [ ]:
samtools view -b NA20538.sam > NA20538_2.bam

This time there is no need for the __`-h`__ option, however we have to tell `samtools` that we want the output in BAM format. We do this by adding the __`-b`__ option. Note we have called the file `NA20538_2.bam` so that the original `NA20538.bam` file is not overwritten.

Check the conversion was successful. 

First check the header section:

In [ ]:
samtools view -h NA20538_2.bam | head -5

Then check the alignment section:

In [ ]:
samtools view NA20538_2.bam | head -5

## BAM/CRAM
The __`samtools view`__ command can also be used to convert betwen BAM and CRAM formats. In the data directory there is a BAM file called `yeast.bam` that was created by aligning _S. cerevisiae_ Illumina data to the reference genome `Saccharomyces_cerevisiae.EF4.68.dna.toplevel.fa`.

To convert a BAM to a CRAM, try 

In [ ]:
samtools view -C -T Saccharomyces_cerevisiae.EF4.68.dna.toplevel.fa -o yeast.cram yeast.bam

Here, we use the __`-C`__ option to tell `samtools` we want the output as CRAM, and the __`-T`__ option to specify what reference file to use for the conversion. We also use the __`-o`__ option to specify the name of the output file.

Have a look at what files were created:

In [ ]:
ls -l

As you can see, this has created an index file for the reference genome called `Saccharomyces_cerevisiae.EF4.68.dna.toplevel.fa.fai` and the CRAM file `yeast.cram`.

Check the conversion was successful:

In [ ]:
samtools view yeast.cram | head -5

### Exercises

__Q1: Since CRAM files use reference-based compression, we expect the CRAM file to be smaller than the BAM file. What is the size of the CRAM file?__  

__Q2: Why do we need to provide the reference genome when converting to CRAM format?__ 

__Q3: Convert the CRAM file back to a BAM file called yeast_2.bam?__ 

## FASTQ to SAM/BAM/CRAM
SAM format is mainly used to store alignment data, however in some cases we may want to store the unaligned data in SAM format and for this we can use the picard tools __`FastqToSam`__ application. [Picard tools](https://broadinstitute.github.io/picard/) is a Java application with a number of useful options for manipulating sequence data. 

To convert the FASTQ files from sequencing run 13681_1_18 to unaligned SAM format, try: 

In [ ]:
picard FastqToSam F1=13681_1_18_1.fastq.gz F2=13681_1_18_2.fastq.gz O=13681_1_18.sam SM=13681_1_18

Here, the __`F1`__ and __`F2`__ options are the paired end FASTQ files, the __`O`__ option is used to specify the name of the output SAM file and __`SM`__ is the sample name which will be stored in the header of the SAM file. There are also multiple other options for specifying what metadata to include in the SAM header. To see all available options, run:

In [ ]:
picard FastqToSam -h

Check the conversion was successful:

In [ ]:
head -5 13681_1_18.sam

From here you can convert the SAM file to BAM and CRAM, as described previously. 

## CRAM to FASTQ
The __`samtools fastq`__ command can be used to convert CRAM to FASTQ directly. However, for many applications we need the FASTQ files ordered so that the order of the reads in the first file match the order of the reads in the second file. This is to allow the reads in a fragment to be matched up based on their location in the FASTQ file. For this reason, we first use __`samtools collate`__ to produce a collated BAM file which ensures that reads of the same name (and therefore from the same fragment) are grouped together in contiguous groups.

In [ ]:
samtools collate yeast.cram yeast.collated

Have a look at what files were created:

In [ ]:
ls

The newly produced BAM file will be called `yeast.collated.bam`. Have a look at the contents and notice how the order of the reads differs from the `yeast.cram` file:

In [ ]:
samtools view yeast.collated.bam | head -6

Let's use this to create two FASTQ files, one for the first reads of the fragment and one for the second reads of the fragment:

In [ ]:
samtools fastq -N -1 yeast.collated_1.fastq -2 yeast.collated_2.fastq yeast.collated.bam

Here, we use the __`-N`__ option to tell `samtools` to include the /1 and /2 in the read name.

Check the conversion was succesful:

In [ ]:
head -4 yeast.collated_1.fastq
head -4 yeast.collated_2.fastq

## VCF/BCF
In a similar way that __`samtools view`__ can be used to convert between SAM, BAM and CRAM, __`bcftools view`__ can be used to convert between VCF and BCF. 

To convert a BCF file to a VCF file try:

In [ ]:
bcftools view -O v -o 1kg.vcf 1kg.bcf

The __`-O`__ option allows us to specify in what format we want the output, compressed BCF (b), uncompressed BCF (u), compressed VCF (z) or uncompressed VCF (v). With the __`-o`__ option we can select the name of the output file. 

Have a look at what files were generated (the options `-lrt` will list the files in reverse chronological order):


In [ ]:
ls -lrt

Check the conversion was successful:

In [ ]:
bcftools view 1kg.vcf | head -10

To convert a VCF file to BCF, try:

In [ ]:
bcftools view -O b -o 1kg_2.bcf 1kg.vcf

Note we have called the file `1kg_2.bcf` so that the original `1kg.bcf` file is not overwritten.

Check the conversion was successful:

In [ ]:
bcftools view 1kg_2.bcf | head -10

### Exercises

__Q4: Convert the BCF file 1kg.bcf to a compressed VCF file called 1kg.vcf.gz__  

__Congratulations__ you have reached the end of the tutorial!